<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2022-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>

<h1 align="center"><font size="5">Supervised Machine Learning: Regression - Final Assignment</font></h1>


## Instructions:

In this Assignment, you will demonstrate the data regression skills you have learned by completing this course. You are expected to leverage a wide variety of tools, but also this report should focus on present findings, insights, and next steps. You may include some visuals from your code output, but this report is intended as a summary of your findings, not as a code review. 

The grading will center around 5 main points:

1. Does the report include a section describing the data?
2. Does the report include a paragraph detailing the main objective(s) of this analysis? 
3. Does the report include a section with variations of linear regression models and specifies which one is the model that best suits the main objective(s) of this analysis.
4. Does the report include a clear and well-presented section with key findings related to the main objective(s) of the analysis?
5. Does the report highlight possible flaws in the model and a plan of action to revisit this analysis with additional data or different predictive modeling techniques? 




## Import the required libraries


The following required modules are pre-installed in the Skills Network Labs environment. However if you run this notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda) you will need to install these libraries by removing the `#` sign before `!mamba` in the code cell below.


In [ ]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !mamba install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 scikit-learn==0.20.1
# Note: If your environment doesn't support "!mamba install", use "!pip install"

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn as skl
import seaborn as sns

## Importing the Dataset


Before you begin, you will need to choose a data set that you feel passionate about. You can brainstorm with your peers about great public data sets using the discussion board in this module.


Read your chosen dataset into pandas dataframe:


In [2]:
data = pd.read_csv('crimedata.csv')
data.head()

,communityName,state,countyCode,communityCode,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,...,burglaries,burglPerPop,larcenies,larcPerPop,autoTheft,autoTheftPerPop,arsons,arsonsPerPop,ViolentCrimesPerPop,nonViolPerPop
0,BerkeleyHeightstownship,NJ,39.0,5320.0,11980,3.10,1.37,91.78,6.50,1.88,...,14.0,114.85,138.0,1132.08,16.0,131.26,2.0,16.41,41.02,1394.59
1,Marpletownship,PA,45.0,47616.0,23123,2.82,0.80,95.57,3.44,0.85,...,57.0,242.37,376.0,1598.78,26.0,110.55,1.0,4.25,127.56,1955.95
2,Tigardcity,OR,NaN,NaN,29344,2.43,0.74,94.33,3.43,2.35,...,274.0,758.14,1797.0,4972.19,136.0,376.30,22.0,60.87,218.59,6167.51
3,Gloversvillecity,NY,35.0,29443.0,16656,2.40,1.70,97.35,0.50,0.70,...,225.0,1301.78,716.0,4142.56,47.0,271.93,NaN,NaN,306.64,NaN
4,Bemidjicity,MN,7.0,5068.0,11245,2.76,0.53,89.16,1.17,0.52,...,91.0,728.93,1060.0,8490.87,91.0,728.93,5.0,40.05,NaN,9988.79


In the EDA notebook, we have already performed an extensive analysis to select the appropriate features for this regression analysis. For the regression analysis, we are only going to concentrate on **predicting violent crime rates** for a given community. We will filter our dataframe by the columns that shown the highest correlation with violent crime rates straight away.

In [10]:
selected_cols = ['PctKidsBornNeverMar', 'nonViolPerPop', 'racepctblack', 'FemalePctDiv',
       'TotalPctDiv', 'pctWPubAsst', 'pctWInvInc', 'PctYoungKids2Par',
       'PctTeen2Par', 'racePctWhite', 'PctFam2Par', 'PctKids2Par',
       'ViolentCrimesPerPop']

In [11]:
df = data.loc[:, [i for i in selected_cols]]
df.columns.unique()

Index(['PctKidsBornNeverMar', 'nonViolPerPop', 'racepctblack', 'FemalePctDiv',
       'TotalPctDiv', 'pctWPubAsst', 'pctWInvInc', 'PctYoungKids2Par',
       'PctTeen2Par', 'racePctWhite', 'PctFam2Par', 'PctKids2Par',
       'ViolentCrimesPerPop'],
      dtype='object')

Once you have selected a data set, you will produce the deliverables listed below and submit them to one of your peers for review. Treat this exercise as an opportunity to produce analysis that are ready to highlight your analytical skills for a senior audience, for example, the Chief Data Officer, or the Head of Analytics at your company.
Sections required in your report:

*   Main objective of the analysis that specifies whether your model will be focused on prediction or interpretation.
*   Brief description of the data set you chose and a summary of its attributes.
*   Brief summary of data exploration and actions taken for data cleaning and feature engineering.
*   Summary of training at least three linear regression models which should be variations that cover using a simple  linear regression as a baseline, adding polynomial effects, and using a regularization regression. Preferably, all use the same training and test splits, or the same cross-validation method.
*  A paragraph explaining which of your regressions you recommend as a final model that best fits your needs in terms of accuracy and explainability.
*  Summary Key Findings and Insights, which walks your reader through the main drivers of your model and insights from your data derived from your linear regression model.
*  Suggestions for next steps in analyzing this data, which may include suggesting revisiting this model adding specific data features to achieve a better explanation or a better prediction.


# 1. About the Data


This is a dataset of 2018 US communities, demographics of each community, and their crime rates. The original dataset has 146 variables where the first four columns are community/location, the middle features are demographic information about each community such as population, age, race, income, and the final columns are types of crimes and overall crime rates.<br>
You can find the original dataset [here](https://www.kaggle.com/datasets/michaelbryantds/crimedata).

In [12]:
df.shape

(2215, 13)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215 entries, 0 to 2214
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PctKidsBornNeverMar  2215 non-null   float64
 1   nonViolPerPop        2118 non-null   float64
 2   racepctblack         2215 non-null   float64
 3   FemalePctDiv         2215 non-null   float64
 4   TotalPctDiv          2215 non-null   float64
 5   pctWPubAsst          2215 non-null   float64
 6   pctWInvInc           2215 non-null   float64
 7   PctYoungKids2Par     2215 non-null   float64
 8   PctTeen2Par          2215 non-null   float64
 9   racePctWhite         2215 non-null   float64
 10  PctFam2Par           2215 non-null   float64
 11  PctKids2Par          2215 non-null   float64
 12  ViolentCrimesPerPop  1994 non-null   float64
dtypes: float64(13)
memory usage: 225.1 KB


As you can see, the each feature of the dataset is of the type float. The features with missing data is *ViolentCrimesPerPop* and *nonViolPerPop*. *ViolentCrimesPerPop* is our predictor variable. We will handle this before we begin our modelling.

In [14]:
df.describe()

,PctKidsBornNeverMar,nonViolPerPop,racepctblack,FemalePctDiv,TotalPctDiv,pctWPubAsst,pctWInvInc,PctYoungKids2Par,PctTeen2Par,racePctWhite,PctFam2Par,PctKids2Par,ViolentCrimesPerPop
count,2215.000000,2118.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,2215.000000,1994.000000
mean,3.115499,4908.241804,9.335102,12.325300,10.812515,6.801445,43.750935,81.865422,75.521788,83.979819,74.059129,71.227255,589.078922
std,3.127681,2739.708901,14.247156,3.262613,3.000883,4.700335,12.787925,12.263736,10.365262,16.419080,10.525952,12.045048,614.784518
min,0.000000,116.790000,0.000000,3.350000,2.830000,0.180000,5.810000,8.700000,20.200000,2.680000,22.970000,18.300000,0.000000
25%,1.070000,2918.070000,0.860000,9.860000,8.575000,3.270000,34.680000,74.780000,70.170000,76.320000,67.900000,63.990000,161.700000
50%,2.040000,4425.450000,2.870000,12.520000,10.900000,5.610000,42.880000,83.990000,76.920000,90.350000,75.030000,72.530000,374.060000
75%,3.910000,6229.280000,11.145000,14.745000,12.985000,9.105000,52.740000,91.675000,82.765000,96.225000,81.900000,80.395000,794.400000
max,27.350000,27119.760000,96.670000,23.920000,22.230000,44.820000,89.040000,100.000000,97.340000,99.630000,93.600000,92.580000,4877.060000


In [16]:
df.isna().sum()

PctKidsBornNeverMar      0
nonViolPerPop           97
racepctblack             0
FemalePctDiv             0
TotalPctDiv              0
pctWPubAsst              0
pctWInvInc               0
PctYoungKids2Par         0
PctTeen2Par              0
racePctWhite             0
PctFam2Par               0
PctKids2Par              0
ViolentCrimesPerPop    221
dtype: int64

# 2. Objectives


This will be a prediction-based analysis. The main objective of this report is to predict the rates of crime in US communities. <br>
This notebook will present a follow-on analysis from the EDA notebook where we performed feature engineering and hypothesis testing as a prelude to this analysis. You can find the EDA notebook [here](https://github.com/Kasheme/Data-Science/blob/main/US-Crime-Communities/US%20Crime%20Notebook%20EDA.ipynb) for reference.

# 3. Data Pre-Processing

Brief summary of data exploration and actions taken for data cleaning and feature engineering.

# 3. Linear Regression Models


Summary of training at least three linear regression models which should be variations that cover using a simple  linear regression as a baseline, adding polynomial effects, and using a regularization regression. Preferably, all use the same training and test splits, or the same cross-validation method.

# 4. Insights and key findings


# 5. Next Steps


## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
